In [7]:
!pip install -q "transformers>=4.51.3" peft sentence-transformers faiss-cpu accelerate bitsandbytes


In [8]:
import os, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

BASE_REPO = "/content/YapayZekaChatbot"
os.listdir(BASE_REPO)


fatal: destination path 'YapayZekaChatbot' already exists and is not an empty directory.


['streamlit.log',
 '.gitattributes',
 'FINE-TUNE',
 'app_compare.py',
 'YapayZekaChatbot',
 'RAG',
 '.gradio',
 'README.md',
 '.git',
 'bilgi_guvenligi_dataset']

In [9]:
import pandas as pd
import numpy as np
import faiss

RAG_DIR = os.path.join(BASE_REPO, "RAG")

CHUNKS_META_PATH = os.path.join(RAG_DIR, "chunks_meta.csv")
INDEX_PATH       = os.path.join(RAG_DIR, "faiss_index.bin")
CHUNK_IDS_PATH   = os.path.join(RAG_DIR, "chunk_ids.npy")

# RAG meta (faiss ile kullanılan sade dosya)
chunks_df = pd.read_csv(CHUNKS_META_PATH)

# RAG index & id map
index     = faiss.read_index(INDEX_PATH)
chunk_ids = np.load(CHUNK_IDS_PATH)

print("chunks_meta satır sayısı:", len(chunks_df))
print("FAISS vektör sayısı:", index.ntotal)
print("chunk_ids şekli:", chunk_ids.shape)

chunks_df.head()


chunks_meta satır sayısı: 247
FAISS vektör sayısı: 247
chunk_ids şekli: (247,)


,chunk_id,filename,section_label,chunk_index,text
0,0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
1,1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.10,0,5.10 Bilginin ve diğer ilgili varlıkların kabu...
2,2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.11,0,5.11 Varlıkların İadesi Personel ve diğer ilgi...
3,3,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.12,0,"5.12 Bilgilerin Sınıflandırılması Bilgiler, ku..."
4,4,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.13,0,5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...


In [10]:
MASTER_PATH = os.path.join(
    BASE_REPO,
    "bilgi_guvenligi_dataset",
    "bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL.csv"
)

master_df = pd.read_csv(MASTER_PATH)
print("master_df satır sayısı:", len(master_df))
master_df.head(), master_df.columns


master_df satır sayısı: 247


(                                         filename  ...                                      chunk_text_tr
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf  ...  5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf  ...  5.10 Bilginin ve diğer ilgili varlıkların kabu...
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf  ...  5.11 Varlıkların İadesi Personel ve diğer ilgi...
 3  ISO-27001-Controls-List-Free-Download-pdf-.pdf  ...  5.12 Bilgilerin Sınıflandırılması Bilgiler, ku...
 4  ISO-27001-Controls-List-Free-Download-pdf-.pdf  ...  5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...
 
 [5 rows x 7 columns],
 Index(['filename', 'source_url', 'section_label', 'section_index',
        'chunk_index', 'chunk_text', 'chunk_text_tr'],
       dtype='object'))

In [11]:
URL_MAP = {}

for _, row in master_df.iterrows():
    key = (row["filename"], row["section_label"], int(row["chunk_index"]))
    URL_MAP[key] = row["source_url"]

print("URL_MAP boyutu:", len(URL_MAP))
list(URL_MAP.items())[:3]


URL_MAP boyutu: 247


[(('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.1', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/'),
 (('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.10', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/'),
 (('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.11', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/')]

In [12]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [13]:
from sentence_transformers import SentenceTransformer

embed_model_name = "intfloat/multilingual-e5-large"  # RAG.ipynb'de ne kullandıysan aynısı
embed_model = SentenceTransformer(embed_model_name, device=device)
embed_model.max_seq_length = 512

embed_model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [40]:
import numpy as np
import torch

TEXT_COL = "text"

RAG_MIN_SCORE   = 0.80  # FAISS skoru bu eşikten küçükse chunk alınmaz
DOMAIN_SAMPLE_SIZE = 256 # alan merkezi için kaç chunk örnekleniyor
DOMAIN_MIN_SIM  = 0.75  # soru bu eşikten düşükse alan dışı kabul edilir

# ---- Alan (domain) merkezini hesapla ----
sample_size = min(DOMAIN_SAMPLE_SIZE, len(chunks_df))
sample_texts = chunks_df.sample(sample_size, random_state=42)[TEXT_COL].tolist()
sample_texts = [f"passage: {t}" for t in sample_texts]

with torch.no_grad():
    domain_embs = embed_model.encode(
        sample_texts,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )

domain_centroid = domain_embs.mean(axis=0, keepdims=True)
domain_centroid = domain_centroid / np.linalg.norm(
    domain_centroid, axis=1, keepdims=True
)


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [41]:
def search_chunks(query: str, top_k: int = 5):
    """
    Soru için en alakalı chunk'ları döndürür.

    1) Soru embedding'i hesaplanır.
    2) Alan merkezine benzerlik (domain_sim) ölçülür.
       - domain_sim < DOMAIN_MIN_SIM → [] döndür (RAG devre dışı).
    3) FAISS ile top_k en yakın vektör bulunur.
    4) Skor < RAG_MIN_SCORE olanlar elenir.
       - Hiçbiri kalmazsa yine [] döner.
    """
    query_text = f"query: {query}"

    with torch.no_grad():
        q_emb = embed_model.encode(
            [query_text],
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )  # (1, dim)

    # 1️⃣ Alan (domain) benzerliği kontrolü
    domain_sim = float((q_emb @ domain_centroid.T)[0][0])
    if domain_sim < DOMAIN_MIN_SIM:
        # Soru bilgi güvenliği alanından uzak → RAG'i tamamen kapat
        return []

    # 2️⃣ FAISS araması
    scores, indices = index.search(q_emb, top_k)
    raw_scores  = scores[0]
    raw_indices = indices[0]

    # 3️⃣ RAG_MIN_SCORE filtresi
    filtered = []
    for score, idx in zip(raw_scores, raw_indices):
        score = float(score)
        if score >= RAG_MIN_SCORE:
            filtered.append((score, int(idx)))

    if not filtered:
        # En yakın komşular bile yeterince benzer değil → RAG devre dışı
        return []

    # 4️⃣ Seçilen chunk'ların metadata'sını doldur
    results = []
    for score, faiss_idx in filtered:
        cid = int(chunk_ids[faiss_idx])
        row = chunks_df[chunks_df["chunk_id"] == cid].iloc[0]

        filename      = row["filename"]
        section_label = row.get("section_label", "Genel")
        chunk_index   = int(row["chunk_index"])
        text          = row[TEXT_COL]

        key = (filename, section_label, chunk_index)
        url = URL_MAP.get(key, None)

        results.append({
            "score": score,
            "chunk_id": cid,
            "chunk_index": chunk_index,
            "section_label": section_label,
            "filename": filename,
            "text": text,
            "url": url
        })

    return results


In [42]:
test_res = search_chunks("Uzaktan çalışma sırasında VPN neden zorunlu tutulmalıdır?", top_k=3)
for i, ch in enumerate(test_res, 1):
    print(f"[{i}] {ch['filename']} | {ch['section_label']} | idx={ch['chunk_index']} | skor={ch['score']:.3f}")
    print("URL:", ch['url'])
    print("Metin:", ch['text'][:120], "...\n")


[1] ISO-27001-Controls-List-Free-Download-pdf-.pdf | ISO_6.7 | idx=0 | skor=0.865
URL: https://hightable.io/iso-27001-annex-a-controls-list/
Metin: 6.7 Uzaktan çalışma Personel, kuruluşun tesisleri dışında erişilen, işlenen veya saklanan bilgileri korumak için uzaktan ...

[2] SANS_Acceptable_Encryption_Standard_April2025.pdf | full | idx=3 | skor=0.844
URL: https://www.sans.org/information-security-policy/acceptable-encryption-standard/
Metin: ve uzaktan erişim ayrıcalıklarının geçici olarak askıya alınmasına ve ciddi durumlarda iş akdinin veya sözleşmeden doğan ...

[3] SANS_Internal_Network_Access_Management_Policy_April2025.pdf | Safeguards | idx=1 | skor=0.838
URL: https://www.sans.org/information-security-policy/internal-network-access-management-policy/
Metin: (802.1x) kablosuz ağlarının her biri için sertifika tabanlı kimlik doğrulama gerektirir. INA-10 Kuruluşun kablosuz ağlar ...



In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL_ID = "google/gemma-2-2b-it"
ADAPTER_PATH  = os.path.join(BASE_REPO, "FINE-TUNE", "gemma-2b-train-qlora-final")

# Tokenizer (adapter klasöründen veya base modelden)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# 1) Base model (saf Gemma)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
base_model.config.pad_token_id = tokenizer.eos_token_id
base_model.eval()

# 2) Fine-tune edilmiş model için ayrı bir base yükleyip LoRA giydiriyoruz
ft_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
ft_model = PeftModel.from_pretrained(
    ft_base,
    ADAPTER_PATH,
)
ft_model.config.pad_token_id = tokenizer.eos_token_id
ft_model.eval()

print("✅ Base ve fine-tune modeller yüklendi.")


KeyboardInterrupt: 

In [43]:
def generate_det(model, tokenizer, prompt: str, max_new_tokens: int = 400):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,   # 🔒 deterministik
        )

    out_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(out_ids, skip_special_tokens=True).strip()
    return text


In [44]:
#Sadece soru (RAG yok)
def build_plain_prompt(question: str) -> str:
    user_content = (
        "Sen bilgi güvenliği ve ISO 27001 alanında soru yanıtlayan bir asistansın. "
        "Elindeki genel bilgiye göre, açık ve teknik ama anlaşılır Türkçe kullanarak cevap ver. "
        "Emin olmadığın konularda uydurma, kısıtlarını belirt.\n\n"
        f"Soru: {question}"
    )

    messages = [
        {"role": "user", "content": user_content}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt


In [45]:
def build_rag_user_content(question: str, retrieved_chunks) -> str:
    if not retrieved_chunks:
        context_text = "İlgili kaynak bulunamadı."
    else:
        blocks = []
        for i, ch in enumerate(retrieved_chunks, 1):
            block = (
                f"[{i}] Kaynak (dosya: {ch['filename']}, bölüm: {ch['section_label']}, chunk_index: {ch['chunk_index']})\n"
                f"{ch['text']}"
            )
            blocks.append(block)
        context_text = "\n\n".join(blocks)

    user_content = (
        "Sen ISO 27001 ve SANS bilgi güvenliği politikaları konusunda uzman bir asistansın. "
        "Aşağıda ilgili politika parçalarını göreceksin.\n\n"
        "Sadece bu kaynaklara dayanarak cevap ver. "
        "Eğer kaynaklar soruyu yanıtlamaya yetmiyorsa veya emin değilsen:\n"
        "- 'Kaynaklarda bu sorunun yanıtı yok.' de\n"
        "- KESİNLİKLE tahmin yürütme veya uydurma.\n\n"
        "=== KAYNAK METİNLER ===\n"
        f"{context_text}\n\n"
        "=== GÖREVİN ===\n"
        "1) Soruyu bu kaynaklara dayanarak yanıtla.\n"
        "2) Kaynaklarda olmayan bilgiyi uydurma, emin değilsen bunu açıkça söyle.\n\n"
        f"=== SORU ===\n{question}"
    )
    return user_content


def build_rag_prompt(question: str, top_k: int = 5):
    """
    RAG prompt'u üretir.
    - Eğer search_chunks hiç sonuç vermezse: (None, []) döndürür.
    """
    retrieved = search_chunks(question, top_k=top_k)

    if not retrieved:
        return None, []

    user_content = build_rag_user_content(question, retrieved)

    messages = [
        {"role": "user", "content": user_content}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    return prompt, retrieved


In [46]:
def run_all_variants(question: str, top_k: int = 5):
    results = []

    # 1) Base model (no RAG)
    plain_prompt = build_plain_prompt(question)
    base_plain_answer = generate_det(base_model, tokenizer, plain_prompt)
    results.append({
        "variant": "A) Base model (no RAG)",
        "answer": base_plain_answer,
        "used_chunks": [],
        "source_links": []
    })

    # 2) Base model + RAG
    base_rag_prompt, retrieved_base = build_rag_prompt(question, top_k=top_k)

    if (base_rag_prompt is None) or (not retrieved_base):
        base_rag_answer = (
            "Bu soru için ilgili ISO/SANS kaynakları bulunamadı. "
            "RAG devreye giremediği için bu model yapılandırması cevap üretmemektedir."
        )
        base_used_chunks = []
        base_links = []
    else:
        base_rag_answer = generate_det(base_model, tokenizer, base_rag_prompt)
        base_used_chunks = retrieved_base
        base_links = sorted({ch["url"] for ch in retrieved_base if ch.get("url")})

    results.append({
        "variant": "B) Base model + RAG",
        "answer": base_rag_answer,
        "used_chunks": base_used_chunks,
        "source_links": base_links
    })

    # 3) Fine-tuned model (no RAG)
    ft_plain_prompt = build_plain_prompt(question)
    ft_plain_answer = generate_det(ft_model, tokenizer, ft_plain_prompt)
    results.append({
        "variant": "C) Fine-tuned model (no RAG)",
        "answer": ft_plain_answer,
        "used_chunks": [],
        "source_links": []
    })

    # 4) Fine-tuned model + RAG
    ft_rag_prompt, retrieved_ft = build_rag_prompt(question, top_k=top_k)

    if (ft_rag_prompt is None) or (not retrieved_ft):
        ft_rag_answer = (
            "Bu soru için ilgili ISO/SANS kaynakları bulunamadı. "
            "RAG devreye giremediği için bu model yapılandırması cevap üretmemektedir."
        )
        ft_used_chunks = []
        ft_links = []
    else:
        ft_rag_answer = generate_det(ft_model, tokenizer, ft_rag_prompt)
        ft_used_chunks = retrieved_ft
        ft_links = sorted({ch["url"] for ch in retrieved_ft if ch.get("url")})

    results.append({
        "variant": "D) Fine-tuned model + RAG",
        "answer": ft_rag_answer,
        "used_chunks": ft_used_chunks,
        "source_links": ft_links
    })

    return results


In [4]:
!pip install --upgrade gradio


In [47]:
import gradio as gr

UI_TO_VARIANT_KEY = {
    "Base model": "A) Base model (no RAG)",
    "Base + RAG": "B) Base model + RAG",
    "Fine-tuned model": "C) Fine-tuned model (no RAG)",
    "Fine-tuned model + RAG": "D) Fine-tuned model + RAG",
}

VARIANT_DISPLAY_NAME = {
    "A) Base model (no RAG)": "Base model",
    "B) Base model + RAG": "Base model + RAG",
    "C) Fine-tuned model (no RAG)": "Fine-tuned model",
    "D) Fine-tuned model + RAG": "Fine-tuned + RAG",
}

def format_md_answer(item):
    if item is None:
        return "_Bu yapılandırma bu çalıştırmada seçilmedi._"

    title = VARIANT_DISPLAY_NAME[item["variant"]]
    answer = (item["answer"] or "").strip()
    links = item.get("source_links", [])
    used_chunks = item.get("used_chunks", [])

    # RAG durumu
    if used_chunks:
        chunk_info = f"{len(used_chunks)} adet chunk kullanıldı."
    else:
        if "+ RAG" in item["variant"]:
            chunk_info = (
                f"Bu soru için yeterince ilgili kaynak bulunamadığından "
                f"RAG devre dışı bırakılmış ve bu yapılandırma cevap üretmemiştir "
                f"(alan eşiği: {DOMAIN_MIN_SIM:.2f}, skor eşiği: {RAG_MIN_SCORE:.2f})."
            )
        else:
            chunk_info = "Bu yapılandırma RAG kullanmıyor."

    if links:
        links_md = "\n".join(f"- {url}" for url in links)
    else:
        links_md = "_Kaynak yok (RAG devrede değil veya URL bulunamadı)._"

    md = f"""### {title}

**Cevap:**

{answer}

---

**Durum:** {chunk_info}

**Kaynaklar:**
{links_md}
"""
    return md

def compare_fn(question, selected_variants, top_k):
    if not question.strip():
        return ("Lütfen soru yazın.", "_", "_", "_", "_")

    if not selected_variants:
        return ("Hiç model seçilmedi.", "_", "_", "_", "_")

    all_results = run_all_variants(
        question=question,
        top_k=int(top_k)
    )

    by_key = {item["variant"]: item for item in all_results}
    selected_keys = {UI_TO_VARIANT_KEY[name] for name in selected_variants}

    def get_item(name):
        v_key = UI_TO_VARIANT_KEY[name]
        return by_key[v_key] if v_key in selected_keys else None

    summary = f"""
### Çalıştırma Özeti

- **Soru:** {question}
- **Seçilen modeller:** {", ".join(selected_variants)}
- **RAG top_k:** {int(top_k)}
"""

    return (
        summary,
        format_md_answer(get_item("Base model")),
        format_md_answer(get_item("Base + RAG")),
        format_md_answer(get_item("Fine-tuned model")),
        format_md_answer(get_item("Fine-tuned model + RAG")),
    )

with gr.Blocks() as demo:
    gr.Markdown(
        "# 🔐 Bilgi Güvenliği LLM Karşılaştırma Paneli\n"
        "Aynı soruyu dört farklı model yapılandırmasıyla çalıştırıp yanıtları yan yana inceleyebilirsin."
    )

    with gr.Row():
        with gr.Column(scale=1):
            question_box = gr.Textbox(
                label="Soru",
                lines=4,
                placeholder="Örn: Güçlü parola politikası nasıl olmalıdır?"
            )

            variant_selector = gr.CheckboxGroup(
                label="Karşılaştırılacak model yapılandırmaları",
                choices=[
                    "Base model",
                    "Base + RAG",
                    "Fine-tuned model",
                    "Fine-tuned model + RAG",
                ],
                value=[
                    "Base model",
                    "Base + RAG",
                    "Fine-tuned model",
                    "Fine-tuned model + RAG",
                ]
            )

            topk_slider = gr.Slider(
                minimum=1,
                maximum=8,
                value=4,
                step=1,                 # 🔹 tam sayı adım
                label="RAG için top_k (kaç chunk getirilsin?)",
            )

            run_btn = gr.Button("Karşılaştırmayı Çalıştır")

        with gr.Column(scale=2):
            summary_md = gr.Markdown()
            with gr.Tab("Base model"):
                base_md = gr.Markdown()
            with gr.Tab("Base + RAG"):
                base_rag_md = gr.Markdown()
            with gr.Tab("Fine-tuned"):
                ft_md = gr.Markdown()
            with gr.Tab("Fine-tuned + RAG"):
                ft_rag_md = gr.Markdown()

    run_btn.click(
        compare_fn,
        inputs=[question_box, variant_selector, topk_slider],
        outputs=[summary_md, base_md, base_rag_md, ft_md, ft_rag_md]
    )

demo.queue().launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://51631dc5b81aff52f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
